In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Import pipeline modules
import sys
sys.path.insert(0, str(Path.cwd()))

from activity_extraction import (
    parse_adl_file, identify_activity_intervals,
    extract_propulsion_activities, extract_resting_activities,
    add_baseline_reference
)
from hr_metrics import (
    extract_rr_intervals_from_ppg, extract_hr_metrics_from_timeseries,
    compute_differential_metrics
)
from window_overlap_analysis import (
    segment_activity_into_phases, extract_phases_from_data,
    create_window_overlap_report
)
from data_loading import load_timeseries_data, extract_window_data, estimate_sampling_frequency

# Set up plotting
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("All libraries imported successfully!")

In [ ]:
# Load PPG signal data
if not PPG_PATH.exists():
    print(f"Warning: PPG file not found at {PPG_PATH}")
    print("Please update PPG_PATH with your data file path")
else:
    ppg_data = load_timeseries_data(PPG_PATH)
    print(f"\nPPG Signal loaded: {len(ppg_data)} samples")
    print(f"Time range: {ppg_data['t_sec'].min():.1f} - {ppg_data['t_sec'].max():.1f} seconds")
    print(f"Value range: {ppg_data['value'].min():.2f} - {ppg_data['value'].max():.2f}")
    
    # Estimate sampling frequency
    fs = estimate_sampling_frequency(ppg_data['t_sec'].values)
    print(f"Estimated sampling frequency: {fs:.2f} Hz")
    
    print(ppg_data.head(10))

In [ ]:
# Extract propulsion and resting activities
propulsion = extract_propulsion_activities(adl_intervals, min_duration_sec=30.0)
resting = extract_resting_activities(adl_intervals, min_duration_sec=60.0)

print(f"Propulsion activities: {len(propulsion)}")
print(f"  Duration range: {propulsion['duration_sec'].min():.1f} - {propulsion['duration_sec'].max():.1f} sec")
print(f"  Mean duration: {propulsion['duration_sec'].mean():.1f} sec")

print(f"\nResting activities: {len(resting)}")
print(f"  Duration range: {resting['duration_sec'].min():.1f} - {resting['duration_sec'].max():.1f} sec")
print(f"  Mean duration: {resting['duration_sec'].mean():.1f} sec")

print(f"\nPropulsion samples:")
print(propulsion.head())

In [ ]:
# Extract resting windows
resting_windows = []

for idx, activity in resting.head(10).iterrows():  # Start with first 10 for exploration
    t_start = activity['t_start']
    t_end = activity['t_end']
    
    signal, time = extract_window_data(ppg_data, t_start, t_end)
    
    if len(signal) >= 100:
        resting_windows.append({
            'resting_idx': idx,
            't_start': t_start,
            't_end': t_end,
            'duration_sec': activity['duration_sec'],
            'n_samples': len(signal),
            'signal': signal,
            'time': time,
        })

print(f"Successfully extracted {len(resting_windows)} resting windows")

if len(resting_windows) > 0:
    # Visualize first resting activity
    window = resting_windows[0]
    plt.figure(figsize=(14, 4))
    plt.plot(window['time'], window['signal'], linewidth=1)
    plt.xlabel('Time (seconds)')
    plt.ylabel('PPG Signal')
    plt.title(f'Example Resting Activity (Duration: {window[\"duration_sec\"]:.1f} sec)')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

In [ ]:
# Compute HR metrics for resting windows
resting_metrics_list = []

for window in resting_windows:
    metrics = extract_hr_metrics_from_timeseries(
        window['signal'], window['time'],
        signal_type='ppg',
        fs=fs,
        compute_frequency_domain=False
    )
    
    result = {
        'resting_idx': window['resting_idx'],
        't_start': window['t_start'],
        't_end': window['t_end'],
        'duration_sec': window['duration_sec'],
    }
    result.update(metrics)
    resting_metrics_list.append(result)

resting_metrics_df = pd.DataFrame(resting_metrics_list)

print("Resting HR Metrics:")
print(resting_metrics_df[['resting_idx', 'mean_hr', 'rmssd', 'sdnn', 'stress_index']])
print(f"\nSummary statistics:")
print(resting_metrics_df[['mean_hr', 'rmssd', 'sdnn', 'stress_index']].describe())

In [ ]:
# Visualize differentials
fig, axes = plt.subplots(2, 2, figsize=(14, 8))

# HR change
axes[0, 0].scatter(differentials_df['resting_mean_hr'], differentials_df['delta_mean_hr'], alpha=0.6)
axes[0, 0].axhline(y=0, color='r', linestyle='--', alpha=0.3)
axes[0, 0].set_xlabel('Resting HR (bpm)')
axes[0, 0].set_ylabel('Δ HR during activity (bpm)')
axes[0, 0].set_title('HR Response to Propulsion')
axes[0, 0].grid(True, alpha=0.3)

# Percent HR change
axes[0, 1].hist(differentials_df['pct_change_mean_hr'].dropna(), bins=15, alpha=0.7, edgecolor='black')
axes[0, 1].set_xlabel('Percent Change in HR (%)')
axes[0, 1].set_ylabel('Count')
axes[0, 1].set_title('Distribution of HR % Change')
axes[0, 1].grid(True, alpha=0.3)

# RMSSD change
axes[1, 0].scatter(differentials_df['resting_rmssd'], differentials_df['delta_rmssd'], alpha=0.6)
axes[1, 0].axhline(y=0, color='r', linestyle='--', alpha=0.3)
axes[1, 0].set_xlabel('Resting RMSSD (ms)')
axes[1, 0].set_ylabel('Δ RMSSD during activity (ms)')
axes[1, 0].set_title('RMSSD (Parasympathetic) Response')
axes[1, 0].grid(True, alpha=0.3)

# Stress index change
axes[1, 1].scatter(differentials_df['resting_stress_index'], differentials_df['delta_stress_index'], alpha=0.6)
axes[1, 1].axhline(y=0, color='r', linestyle='--', alpha=0.3)
axes[1, 1].set_xlabel('Resting Stress Index')
axes[1, 1].set_ylabel('Δ Stress Index during activity')
axes[1, 1].set_title("Baevsky's Stress Response")
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nDifferential Summary Statistics:")
print(differentials_df[['delta_mean_hr', 'pct_change_mean_hr', 'delta_rmssd', 'delta_stress_index']].describe())

In [ ]:
# Compute HR metrics for each phase
from hr_metrics import extract_rr_intervals_from_ppg, compute_hr_metrics_for_window

phase_metrics = {}

for phase_name, phase_df in phase_data.items():
    if phase_df is None or len(phase_df) < 100:
        phase_metrics[phase_name] = None
        continue
    
    signal = phase_df['value'].values
    rr_intervals, _ = extract_rr_intervals_from_ppg(signal, fs=fs)
    
    if len(rr_intervals) >= 5:
        metrics = compute_hr_metrics_for_window(rr_intervals)
        phase_metrics[phase_name] = metrics
    else:
        phase_metrics[phase_name] = None

print("HR Metrics by Activity Phase:")
for phase_name, metrics in phase_metrics.items():
    if metrics is not None:
        print(f"\n{phase_name}:")
        print(f"  Mean HR: {metrics.get('mean_hr', np.nan):.1f} bpm")
        print(f"  RMSSD: {metrics.get('rmssd', np.nan):.1f} ms (vagal/parasympathetic)")
        print(f"  Stress Index: {metrics.get('stress_index', np.nan):.1f}")
    else:
        print(f"\n{phase_name}: No metrics (insufficient beats detected)")

In [ ]:
# Visualize activity with baseline and recovery phases
fig, axes = plt.subplots(2, 1, figsize=(16, 8))

# Extract full window with context
t_start_context = phases['baseline'][0]
t_end_context = phases['recovery_complete'][1]

mask = (ppg_data['t_sec'] >= t_start_context) & (ppg_data['t_sec'] <= t_end_context)
window_context = ppg_data[mask].copy()
window_context['t_relative'] = window_context['t_sec'] - sample_activity['t_start']

# Plot raw signal
axes[0].plot(window_context['t_relative'], window_context['value'], linewidth=1, label='PPG Signal', alpha=0.7)

# Add phase boundaries
for phase_name, (p_start, p_end) in phases.items():
    t_rel_start = p_start - sample_activity['t_start']
    t_rel_end = p_end - sample_activity['t_start']
    
    if phase_name == 'activity':
        axes[0].axvspan(t_rel_start, t_rel_end, alpha=0.2, color='red', label='Activity')
    elif phase_name == 'baseline':
        axes[0].axvspan(t_rel_start, t_rel_end, alpha=0.2, color='green', label='Baseline')
    else:
        axes[0].axvspan(t_rel_start, t_rel_end, alpha=0.1, color='blue', label=phase_name if phase_name not in ['recovery_immediate', 'recovery_extended'] else '')

axes[0].set_ylabel('PPG Signal Amplitude')
axes[0].set_title(f'Activity Phase Timeline: {sample_activity.get(\"activity\", \"Activity\")}')
axes[0].legend(loc='upper right')
axes[0].grid(True, alpha=0.3)

# Plot normalized signal for easier visualization
window_context['value_norm'] = (window_context['value'] - window_context['value'].mean()) / window_context['value'].std()
axes[1].plot(window_context['t_relative'], window_context['value_norm'], linewidth=1, label='Normalized PPG', alpha=0.7)

# Add phase boundaries
for phase_name, (p_start, p_end) in phases.items():
    t_rel_start = p_start - sample_activity['t_start']
    t_rel_end = p_end - sample_activity['t_start']
    
    if phase_name == 'activity':
        axes[1].axvspan(t_rel_start, t_rel_end, alpha=0.2, color='red')
    elif phase_name == 'baseline':
        axes[1].axvspan(t_rel_start, t_rel_end, alpha=0.2, color='green')
    else:
        axes[1].axvspan(t_rel_start, t_rel_end, alpha=0.1, color='blue')

axes[1].axhline(y=0, color='k', linestyle='-', linewidth=0.5, alpha=0.3)
axes[1].set_xlabel('Time relative to activity start (seconds)')
axes[1].set_ylabel('Normalized PPG Signal')
axes[1].set_title('Normalized Signal: Easier to See Phase Transitions')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Save comprehensive results\nsummary_stats = {\n    'n_propulsion_activities': len(propulsion),\n    'n_resting_activities': len(resting),\n    'n_propulsion_with_metrics': len(propulsion_metrics_df),\n    'n_resting_with_metrics': len(resting_metrics_df),\n    'n_comparisons': len(differentials_df),\n    'mean_propulsion_hr': propulsion_metrics_df['mean_hr'].mean() if len(propulsion_metrics_df) > 0 else np.nan,\n    'mean_resting_hr': resting_metrics_df['mean_hr'].mean() if len(resting_metrics_df) > 0 else np.nan,\n    'mean_hr_increase': differentials_df['delta_mean_hr'].mean() if len(differentials_df) > 0 else np.nan,\n    'mean_pct_hr_increase': differentials_df['pct_change_mean_hr'].mean() if len(differentials_df) > 0 else np.nan,\n}\n\nsummary_df = pd.DataFrame([summary_stats])\nsummary_df.to_csv(OUTPUT_DIR / 'analysis_summary.csv', index=False)\n\nprint(\"\\nAnalysis Summary:\")\nfor key, value in summary_stats.items():\n    if isinstance(value, float):\n        print(f\"  {key}: {value:.2f}\")\n    else:\n        print(f\"  {key}: {value}\")\n\nprint(f\"\\nResults saved to {OUTPUT_DIR}/\")"

In [ ]:
# Analyze patterns across multiple activities
print(\"=\"*80)\nprint(\"OPTIMAL WINDOW RECOMMENDATIONS FOR HR METRIC EXTRACTION\")\nprint(\"=\"*80)\n\nif len(phase_metrics) > 0:\n    print(\"\\n1. BASELINE PHASE (2 minutes before activity):\")\n    baseline_m = phase_metrics.get('baseline')\n    if baseline_m:\n        print(f\"   Mean HR: {baseline_m.get('mean_hr', np.nan):.1f} bpm\")\n        print(f\"   RMSSD: {baseline_m.get('rmssd', np.nan):.1f} ms (parasympathetic tone)\")\n        print(f\"   Stress Index: {baseline_m.get('stress_index', np.nan):.1f}\")\n        print(f\"   → Use this as reference for determining activity response\")\n    \n    print(\"\\n2. ACTIVITY PHASE (during the activity):\")\n    activity_m = phase_metrics.get('activity')\n    if activity_m:\n        print(f\"   Mean HR: {activity_m.get('mean_hr', np.nan):.1f} bpm\")\n        print(f\"   RMSSD: {activity_m.get('rmssd', np.nan):.1f} ms\")\n        print(f\"   Stress Index: {activity_m.get('stress_index', np.nan):.1f}\")\n        \n        if baseline_m:\n            hr_increase = activity_m.get('mean_hr', np.nan) - baseline_m.get('mean_hr', np.nan)\n            rmssd_change = activity_m.get('rmssd', np.nan) - baseline_m.get('rmssd', np.nan)\n            print(f\"   HR increase during activity: {hr_increase:+.1f} bpm ({hr_increase/(baseline_m.get('mean_hr', 1))*100:+.1f}%)\")\n            print(f\"   RMSSD change: {rmssd_change:+.1f} ms (negative = reduced parasympathetic)\")\n    \n    print(\"\\n3. IMMEDIATE RECOVERY (0-5 minutes post-activity):\")\n    imm_recovery_m = phase_metrics.get('recovery_immediate')\n    if imm_recovery_m and baseline_m:\n        print(f\"   Mean HR: {imm_recovery_m.get('mean_hr', np.nan):.1f} bpm\")\n        print(f\"   HR change from baseline: {imm_recovery_m.get('mean_hr', np.nan) - baseline_m.get('mean_hr', np.nan):+.1f} bpm\")\n        print(f\"   RMSSD: {imm_recovery_m.get('rmssd', np.nan):.1f} ms\")\n        print(f\"   → HRV may still be reduced; recovery ongoing\")\n    \n    print(\"\\n4. EXTENDED RECOVERY (0-10 minutes post-activity):\")\n    ext_recovery_m = phase_metrics.get('recovery_extended')\n    if ext_recovery_m and baseline_m:\n        print(f\"   Mean HR: {ext_recovery_m.get('mean_hr', np.nan):.1f} bpm\")\n        print(f\"   HR change from baseline: {ext_recovery_m.get('mean_hr', np.nan) - baseline_m.get('mean_hr', np.nan):+.1f} bpm\")\n        print(f\"   RMSSD: {ext_recovery_m.get('rmssd', np.nan):.1f} ms\")\n        print(f\"   → Better parasympathetic recovery expected\")\n\nprint(\"\\n\" + \"=\"*80)\nprint(\"KEY FINDINGS & RECOMMENDATIONS:\")\nprint(\"=\"*80)\nprint(\"\"\"\n1. EXPECTED HR RESPONSE TO PROPULSION:\n   - HR typically increases within 10-30 seconds of activity onset\n   - Peak HR may be reached 1-3 minutes into activity\n   - Recovery: HR returns to baseline over 3-10 minutes post-activity\n\n2. HRV/RMSSD PATTERN:\n   - RMSSD typically DECREASES during activity (reduced parasympathetic tone)\n   - Faster HRV recovery indicates better fitness/recovery\n   - Use RMSSD change as marker of physiological stress\n\n3. STRESS INDEX (Baevsky):\n   - Increases with activity (sympathetic activation)\n   - SI > 150 indicates high physiological stress\n   - Return to baseline SI indicates recovery\n\n4. OPTIMAL WINDOWS FOR ANALYSIS:\n   a) Activity HR response: Full activity window (captures peak response)\n   b) Recovery assessment: 3-5 minutes post-activity minimum\n   c) Baseline stability: 2 minutes pre-activity (good for physiological baseline)\n   d) Long-term recovery: 10-15 minutes for complete assessment\n\n5. NEXT STEPS FOR YOUR PIPELINE:\n   - Validate delay patterns across more activities\n   - Optimize baseline window (currently 2 min; test 1, 2, 3, 5 min)\n   - Explore individual variability in response timing\n   - Consider activity intensity as modulator of response speed\n\"\"\")\n"

## Section 10: Determine Optimal Observation Windows

Analyze delay patterns and provide recommendations for optimal HR metric extraction windows.

In [ ]:
# Create window overlap report for sample activity
activity_dict = sample_activity.to_dict()

overlap_report = create_window_overlap_report(
    activity_dict,
    phases,
    ppg_data,
    hr_time_col='t_sec',
    hr_metric_col='value'
)

print("Window Overlap Report:")
print(overlap_report[[
    'phase', 'phase_duration_sec', 'n_samples', 'metric_mean', 'metric_std'
]])

# Save report
overlap_report.to_csv(OUTPUT_DIR / 'sample_overlap_analysis.csv', index=False)

## Section 9: Visualize Activity Windows and HR Data

Create plots showing activity windows overlaid with HR data to visualize response patterns and delays.

In [ ]:
# Visualize activity phases
fig, axes = plt.subplots(len(phase_data), 1, figsize=(14, 3 * len(phase_data)))

if len(phase_data) == 1:
    axes = [axes]

for idx, (phase_name, phase_df) in enumerate(phase_data.items()):
    if phase_df is not None and len(phase_df) > 0:
        axes[idx].plot(phase_df['phase_time'], phase_df['value'], linewidth=1, alpha=0.7)
        axes[idx].set_ylabel('PPG Signal')
        axes[idx].set_title(f'Phase: {phase_name} ({len(phase_df)} samples)')
        axes[idx].grid(True, alpha=0.3)
    else:
        axes[idx].text(0.5, 0.5, 'No data', ha='center', va='center')
        axes[idx].set_title(f'Phase: {phase_name}')
    
    if idx == len(phase_data) - 1:
        axes[idx].set_xlabel('Time since phase start (seconds)')

plt.tight_layout()
plt.show()

In [ ]:
# For delay analysis, we'll segment activities into phases:
# 1. Baseline: 2 minutes before activity start
# 2. Activity: during the activity
# 3. Recovery: 5 minutes after activity ends

sample_activity = propulsion.iloc[0]

phases = segment_activity_into_phases(
    (sample_activity['t_start'], sample_activity['t_end']),
    baseline_before_sec=120.0,
    recovery_after_sec=300.0
)

print("Activity Phase Definition:")
for phase_name, (t_start, t_end) in phases.items():
    duration = t_end - t_start
    print(f"  {phase_name:20s}: {t_start:10.1f} - {t_end:10.1f} ({duration:6.1f} sec)")

# Extract phase data
phase_data = extract_phases_from_data(
    ppg_data, phases,
    time_col='t_sec',
    signal_col='value'
)

print(f"\nExtracted phase data:")
for phase_name, phase_df in phase_data.items():
    if phase_df is not None:
        print(f"  {phase_name:20s}: {len(phase_df):5d} samples")
    else:
        print(f"  {phase_name:20s}: No data")

## Section 8: Analyze HR Response Delays

Analyze temporal relationship between activity onset and physiological HR response, identifying delays and lag patterns.

In [ ]:
# Compute differentials for paired activities
differentials_list = []

for prop_idx, activity in propulsion_with_baseline.iterrows():
    if pd.isna(activity.get('baseline_t_start')):
        continue
    
    # Find corresponding metrics
    prop_metrics = propulsion_metrics_df[propulsion_metrics_df['activity_idx'] == prop_idx]
    
    # Find resting metrics in baseline window
    rest_metrics = resting_metrics_df[
        (resting_metrics_df['t_start'] >= activity['baseline_t_start']) &
        (resting_metrics_df['t_end'] <= activity['baseline_t_end'])
    ]
    
    if len(prop_metrics) == 0 or len(rest_metrics) == 0:
        continue
    
    prop_m = prop_metrics.iloc[0].to_dict()
    rest_m = rest_metrics.iloc[0].to_dict()
    
    # Compute differentials
    diff_metrics = compute_differential_metrics(prop_m, rest_m)
    
    comparison = {
        'activity_idx': prop_idx,
        'activity': activity['activity'],
        'propulsion_mean_hr': prop_m.get('mean_hr'),
        'resting_mean_hr': rest_m.get('mean_hr'),
        'propulsion_rmssd': prop_m.get('rmssd'),
        'resting_rmssd': rest_m.get('rmssd'),
        'propulsion_stress_index': prop_m.get('stress_index'),
        'resting_stress_index': rest_m.get('stress_index'),
    }
    comparison.update(diff_metrics)
    differentials_list.append(comparison)

differentials_df = pd.DataFrame(differentials_list)

print(f"\nActivity-Baseline Differentials ({len(differentials_df)} pairs):")
print(differentials_df[[
    'propulsion_mean_hr', 'resting_mean_hr', 'delta_mean_hr', 'pct_change_mean_hr',
    'propulsion_rmssd', 'resting_rmssd', 'delta_rmssd', 'pct_change_rmssd'
]].head(10))

In [ ]:
# Pair propulsion with baseline resting periods
propulsion_with_baseline = add_baseline_reference(propulsion, resting)

print(f"Paired {propulsion_with_baseline['baseline_t_start'].notna().sum()} propulsion activities with baselines")
print(propulsion_with_baseline[['activity', 'duration_sec', 'baseline_time_before_sec']].head(10))

## Section 7: Calculate Activity Differentials

Compute differences between propulsion and resting metrics to quantify physiological response to activity.

In [ ]:
# Compute HR metrics for propulsion windows
propulsion_metrics_list = []

for window in propulsion_windows:
    metrics = extract_hr_metrics_from_timeseries(
        window['signal'], window['time'],
        signal_type='ppg',
        fs=fs,
        compute_frequency_domain=False
    )
    
    result = {
        'activity_idx': window['activity_idx'],
        't_start': window['t_start'],
        't_end': window['t_end'],
        'duration_sec': window['duration_sec'],
    }
    result.update(metrics)
    propulsion_metrics_list.append(result)

propulsion_metrics_df = pd.DataFrame(propulsion_metrics_list)

print("Propulsion HR Metrics:")
print(propulsion_metrics_df[['activity_idx', 'mean_hr', 'rmssd', 'sdnn', 'stress_index']])
print(f"\nSummary statistics:")
print(propulsion_metrics_df[['mean_hr', 'rmssd', 'sdnn', 'stress_index']].describe())

## Section 6: Compute HR Metrics

Calculate HR metrics including raw HR, RMSSD, and Baevsky's stress index for both propulsion and resting windows.

## Section 5: Extract Resting Activity Windows

Extract resting activity windows to serve as baseline for HR metric comparison.

In [ ]:
# Extract propulsion windows
propulsion_windows = []

for idx, activity in propulsion.head(10).iterrows():  # Start with first 10 for exploration
    t_start = activity['t_start']
    t_end = activity['t_end']
    
    signal, time = extract_window_data(ppg_data, t_start, t_end)
    
    if len(signal) >= 100:
        propulsion_windows.append({
            'activity_idx': idx,
            't_start': t_start,
            't_end': t_end,
            'duration_sec': activity['duration_sec'],
            'n_samples': len(signal),
            'signal': signal,
            'time': time,
        })

print(f"Successfully extracted {len(propulsion_windows)} propulsion windows")

if len(propulsion_windows) > 0:
    # Visualize first propulsion activity
    window = propulsion_windows[0]
    plt.figure(figsize=(14, 4))
    plt.plot(window['time'], window['signal'], linewidth=1)
    plt.xlabel('Time (seconds)')
    plt.ylabel('PPG Signal')
    plt.title(f'Example Propulsion Activity (Duration: {window[\"duration_sec\"]:.1f} sec)')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

## Section 4: Extract Propulsion Activity Windows

Extract time windows for propulsion activities to prepare for HR metric computation.

In [ ]:
# Identify activity intervals from events
adl_intervals = identify_activity_intervals(adl_df)
print(f"Identified {len(adl_intervals)} activity intervals")
print(f"\nActivity types:")
print(adl_intervals['activity'].value_counts().head(15))
print(f"\nDuration statistics (seconds):")
print(adl_intervals['duration_sec'].describe())

## Section 3: Filter and Identify Activity Types

Identify activity intervals and separate different activity types including propulsion and resting.

In [ ]:
# Configure file paths - UPDATE THESE WITH YOUR DATA
ADL_PATH = Path('data/adl_log.csv')  # ADL (Activities of Daily Living) file
PPG_PATH = Path('data/ppg_signal.csv')  # PPG signal file
OUTPUT_DIR = Path('output')
OUTPUT_DIR.mkdir(exist_ok=True)

# Load ADL data
if not ADL_PATH.exists():
    print(f"Warning: ADL file not found at {ADL_PATH}")
    print("Please update ADL_PATH with your data file path")
else:
    adl_df = parse_adl_file(ADL_PATH)
    print(f"ADL Data loaded: {len(adl_df)} events")
    print(adl_df.head(10))

## Section 2: Load and Explore Data

Load the activity and HR data from files, display basic information about the dataset structure.

## Section 1: Import Required Libraries

Import necessary libraries for data analysis, visualization, and HR metric computation.

# Data Inspection Pipeline - Interactive Analysis

This notebook provides interactive analysis of activity data, HR metrics, and physiological responses.

**Key objectives:**
1. Extract propulsion and resting activities
2. Compute HR metrics (HR, RMSSD, Baevsky stress index)
3. Analyze baseline-activity differentials
4. Identify HR response delays
5. Find optimal observation windows